In [163]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pickle
import seaborn as sns
import pandas as pd
from scipy import stats
import matplotlib.ticker as ticker
from statannotations.Annotator import Annotator

plt.rcParams.update({'font.size': 12})

# Sliding window

In [136]:
def load_df(path):
    # Load sliding window data
    sliding_df = pd.read_csv(path, sep="\t", header=0)
    sliding_df = sliding_df.stack().reset_index()
    sliding_df = sliding_df.rename(columns={'level_0': 'subject', 'level_1': 'Time (ms)', 0: 'Validation accuracy'})

    model_type = []
    for i in range(len(sliding_df)):
        if 'pca' in sliding_df['Time (ms)'][i]:
            model_type.append('LDA-PCA')
            sliding_df['Time (ms)'][i] = sliding_df['Time (ms)'][i].replace(' (pca-wavelet)', '')
        else:
            model_type.append('LDA-NN')
    sliding_df['model type'] = model_type

    # convert Time (ms) to int
    sliding_df['Time (ms)'] = sliding_df['Time (ms)'].astype(int)

    return sliding_df

In [153]:
def plot_df(ax, sliding_df, df, chance):
    p_values = []
    p_conv = []
    p_pca = []
    for t in range(45, 940, 10):
        test1 = sliding_df['Validation accuracy'][(sliding_df['Time (ms)'] == t) & (sliding_df['model type'] == 'LDA-PCA')]
        test2 = sliding_df['Validation accuracy'][(sliding_df['Time (ms)'] == t) & (sliding_df['model type'] == 'LDA-NN')]
        
        p_values.append(stats.wilcoxon(test1, test2)[1] * 90)

        test1 = sliding_df['Validation accuracy'][(sliding_df['Time (ms)'] == t) & (sliding_df['model type'] == 'LDA-NN')]
        test2 = df['Validation accuracy'][df['model']=='LDA-NN']
        
        p_conv.append(stats.wilcoxon(test1, test2)[1] * 90)

        test1 = sliding_df['Validation accuracy'][(sliding_df['Time (ms)'] == t) & (sliding_df['model type'] == 'LDA-PCA')]
        test2 = df['Validation accuracy'][df['model']=='LDA-PCA']
        
        p_pca.append(stats.wilcoxon(test1, test2)[1] * 90)

    
    dots = np.array(range(45, 940, 10))[np.array(p_values)<0.05]
    ax.plot(dots, [0.5]*len(dots), 'ro', markersize=2, label='LDA-NN SW > LDA-PCA SW (p<0.05)')

    # lines for full-epoch accuracy
    dots = np.array(range(45, 940, 10))[np.array(p_conv)<0.05]
    avg = np.average(df['Validation accuracy'][df['model']=='LDA-NN'])
    ax.plot(dots, [avg]*len(dots), 'bo', markersize=2, label='LDA-NN FE > LDA-NN SW (p<0.05)')

    dots = np.array(range(45, 940, 10))[np.array(p_pca)<0.05]
    avg = np.average(df['Validation accuracy'][df['model']=='LDA-PCA'])
    ax.plot(dots, [avg]*len(dots), 'o', color='orange', markersize=2, label='LDA-PCA FE > LDA-PCA SW (p<0.05)')

    # Plot sliding window data, with legend inside the plot and sparse x ticks
    sliding_plot = sns.lineplot(data=sliding_df,
                                x="Time (ms)",
                                y="Validation accuracy",
                                hue='model type',
                                legend='full',
                                n_boot=100,
                                ax=ax)

    ax.set_xlim(0, None)

    if chance>0.01:
        ax.set_ylabel('')

    ax.axhline(chance, ls='--', color='black', label='chance level')

    #ax.legend(loc='upper right', bbox_to_anchor=(1, 1))

In [137]:
cichy118_df = load_df('cichy118_sliding.txt')
cichy92_df = load_df('cichy118_sliding.txt')

/var/folders/nv/wjmf18wd5_j38vg9v0cthl5h0000gn/T/ipykernel_5281/3657271187.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliding_df['Time (ms)'][i] = sliding_df['Time (ms)'][i].replace(' (pca-wavelet)', '')


In [150]:
axes[0].get_legend_handles_labels()

([<matplotlib.lines.Line2D at 0x7fb03b7c64f0>,
 ['LDA-NN SW > LDA-PCA SW (p<0.05)',
  'LDA-NN FE > LDA-NN SW (p<0.05)',
  'LDA-PCA FE > LDA-PCA SW (p<0.05)',
  'LDA-NN',
  'LDA-PCA',
  'chance level'])

In [165]:
%matplotlib widget
fig, axes = plt.subplots(1, 2)

plot_df(axes[0], cichy118_df, df, 0.008)
plot_df(axes[1], cichy92_df, df, 0.011)

handles, labels = axes[0].get_legend_handles_labels()

# remove legend of axes
axes[0].legend_.remove()
axes[1].legend_.remove()
fig.legend(handles,
           labels,
           loc='upper center',
           bbox_to_anchor=(0.5, 1),
           fancybox=True,
           shadow=True,
           ncol=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/ricsi/miniconda3/envs/main/lib/python3.8/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/ricsi/miniconda3/envs/main/lib/python3.8/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [121]:




e=0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Full epoch

In [85]:
df = pd.read_csv('cichy118_epoch.txt', sep="\t", header=0)
df = df.stack().reset_index()
df = df.rename(columns={'level_0': 'subject', 'level_1': 'model', 0: 'Validation accuracy'})

In [72]:
%matplotlib widget

plt.rcParams.update({'font.size': 14})
# Putting the parameters in a dictionary avoids code duplication
# since we use the same for `sns.boxplot` and `Annotator` calls
plot_params = {
    'kind':    'violin',
    'aspect':  1,
    'cut':     0,
    'ci':      None,
    'scale':   'width',
    'data':    df,
    'x':       'model',
    'y':       'Validation accuracy',
    'legend':  None
}

g = sns.catplot(**plot_params)

ax = g.axes[0][0]
ax.axhline(0.011, ls='--', color='black', label='chance')
plt.ylim(0, 1)
plt.xlabel('')
#plt.text(2.1,0.015,'chance')

# which pairs to computer stats on
pairs = [('LDA-PCA', 'LDA-NN')]

# Add statistics annotations
annotator = Annotator(ax, pairs, data=df, x='model', y='Validation accuracy')
annotator.configure(test='Wilcoxon', verbose=True, line_offset_to_group=10).apply_and_annotate()
#plt.savefig('/Users/ricsi/GoogleDrive/Projektek/The Brain Project/oxford/writings/mvpa_figures/cichy118_epoch.pdf', format='pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

p-value annotation legend:
      ns: p <= 1.00e+00
       *: 1.00e-02 < p <= 5.00e-02
      **: 1.00e-03 < p <= 1.00e-02
     ***: 1.00e-04 < p <= 1.00e-03
    ****: p <= 1.00e-04

LDA-PCA vs. LDA-NN: Wilcoxon test (paired samples), P_val:6.104e-05 Stat=0.000e+00


(<AxesSubplot:ylabel='Validation accuracy'>,
 [<statannotations.Annotation.Annotation at 0x7fb04b14bd00>])